# Демонстрация использования класса `FileOrganizer`

Этот ноутбук показывает, как использовать класс `FileOrganizer` из `filesystem.organizer` для выполнения различных операций по организации файлов осциллограмм:
- Обновление времени модификации файлов.
- Извлечение информации о частоте сети и дискретизации из `.cfg` файлов.
- Группировка файлов осциллограмм по этим частотам.
- Создание инвентарного CSV-файла с хеш-суммами `.dat` файлов.

**Важно**: Этот ноутбук использует образцы файлов из директории `tests/sample_data/`. Для демонстрации файлы копируются во временную рабочую директорию (`temp_organizer_notebook_data/source_oscillograms_for_grouping`). Убедитесь, что тестовые данные были сгенерированы запуском скрипта `tests/test_data_setup.py`.

In [ ]:
import os
import shutil
import datetime
import pandas as pd # For displaying CSV content
import numpy as np # For any sample data generation if needed, though ideally not
import sys

# --- Пути к данным для тестирования ---
module_path = os.path.abspath(os.path.join('..')) 
if module_path not in sys.path:
    sys.path.append(module_path)
    
from filesystem.organizer import FileOrganizer # Import after sys.path modification

base_sample_data_dir = os.path.join(module_path, "tests", "sample_data")
comtrade_samples_path = os.path.join(base_sample_data_dir, "comtrade_files")

# --- Создание временной рабочей директории для этого ноутбука ---
notebook_base_temp_dir = "temp_organizer_notebook_data"
# This will be the main directory passed to FileOrganizer methods like group_files or generate_inventory
notebook_source_dir = os.path.join(notebook_base_temp_dir, "source_oscillograms_for_grouping") 
    
if os.path.exists(notebook_base_temp_dir):
    shutil.rmtree(notebook_base_temp_dir)
os.makedirs(notebook_source_dir) # Create the source dir where files will be copied/grouped

# --- Копирование необходимых файлов для демонстрации ---
# Файлы из tests/sample_data/comtrade_files/
# 1. sample_A.cfg/dat (from valid_cfg_dat_1_subdir) -> 50Hz, 1000sps
shutil.copy(os.path.join(comtrade_samples_path, "valid_cfg_dat_1_subdir", "sample_A.cfg"), notebook_source_dir)
shutil.copy(os.path.join(comtrade_samples_path, "valid_cfg_dat_1_subdir", "sample_A.dat"), notebook_source_dir)

# 2. cp1251_encoded.cfg/dat (from encoding_tests) -> Assuming 50Hz, 1500sps for simplicity or specific values if set up
#    (Ensure test_data_setup.py creates cp1251_encoded.cfg with these frequencies)
shutil.copy(os.path.join(comtrade_samples_path, "encoding_tests", "cp1251_encoded.cfg"), notebook_source_dir)
shutil.copy(os.path.join(comtrade_samples_path, "encoding_tests", "cp1251_encoded.dat"), notebook_source_dir)
    
# 3. malformed_cfg_1.cfg (from malformed_cfg_files)
shutil.copy(os.path.join(comtrade_samples_path, "malformed_cfg_files", "malformed_cfg_1.cfg"), notebook_source_dir)
# (No DAT needed for malformed CFG if only frequency extraction is tested on it)

# 4. A CFG file without DAT for testing grouping robustness
with open(os.path.join(notebook_source_dir, "cfg_no_dat.cfg"), "w", encoding="utf-8") as f:
    # Simplified CFG content
    f.write("DummyStation,NoDatDevice,1999\n1,1A,0D\n1,NoDatSig,,V,1,0,0,0,100,1,1,P\n50.0\n1\n1000,10\n01/01/2023,00:00:00.000\n01/01/2023,00:00:01.000\nASCII\n1\n")
        
# 5. A file with different frequencies for grouping, e.g., 60Hz, 2000sps
#    (Ensure test_data_setup.py creates a file like 'sample_B_60_2k.cfg/dat')
sample_B_path = os.path.join(comtrade_samples_path, "for_grouping_B") # Assume a subdir for_grouping_B
if os.path.exists(os.path.join(sample_B_path, "sample_B_60_2k.cfg")):
    shutil.copy(os.path.join(sample_B_path, "sample_B_60_2k.cfg"), notebook_source_dir)
    shutil.copy(os.path.join(sample_B_path, "sample_B_60_2k.dat"), notebook_source_dir)
    print(f"Copied sample_B_60_2k from {sample_B_path}")
else:
    print(f"Warning: sample_B_60_2k.cfg not found in {sample_B_path}. Grouping test might be less effective.")

print(f"Демонстрационная директория для FileOrganizer: {notebook_source_dir}")
print("Содержимое:")
for item in os.listdir(notebook_source_dir): print(f"- {item}")

## 1. Инициализация `FileOrganizer`

In [ ]:
organizer = FileOrganizer()
# Для более подробного вывода можно передать is_print_message=True в методы ниже

## 2. Обновление времени модификации файлов (`update_modification_times`)

In [ ]:
import time

print("Время модификации до обновления:")
for item in os.listdir(notebook_source_dir):
    stat_info = os.stat(os.path.join(notebook_source_dir, item))
    print(f"- {item}: {datetime.datetime.fromtimestamp(stat_info.st_mtime)}")

print("\nОбновляем время модификации...")
# Установим конкретное время для наглядности
custom_datetime = datetime.datetime(2020, 1, 1, 12, 0, 0)
organizer.update_modification_times(notebook_source_dir, new_mod_time=custom_datetime)

print("\nВремя модификации после обновления:")
for item in os.listdir(notebook_source_dir):
    stat_info = os.stat(os.path.join(notebook_source_dir, item))
    print(f"- {item}: {datetime.datetime.fromtimestamp(stat_info.st_mtime)}")

## 3. Извлечение частот из `.cfg` файла (`extract_frequencies_from_cfg`)

In [ ]:
cfg_path_sample_A = os.path.join(notebook_source_dir, "sample_A.cfg")
cfg_path_malformed = os.path.join(notebook_source_dir, "malformed_cfg_1.cfg")
cfg_path_cp1251 = os.path.join(notebook_source_dir, "cp1251_encoded.cfg")

print(f"Извлечение частот из {os.path.basename(cfg_path_sample_A)}:")
freq_net1, freq_rate1 = organizer.extract_frequencies_from_cfg(cfg_path_sample_A, encoding='utf-8', threshold=0.1)
print(f"  Частота сети: {freq_net1} Гц, Частота дискретизации: {freq_rate1} Гц (Ожидается: 50, 1000)")

print(f"\nИзвлечение частот из {os.path.basename(cfg_path_malformed)} (ожидаются ошибки/None):")
freq_net_mal, freq_rate_mal = organizer.extract_frequencies_from_cfg(cfg_path_malformed, encoding='utf-8')
print(f"  Частота сети: {freq_net_mal}, Частота дискретизации: {freq_rate_mal}")

print(f"\nИзвлечение частот из {os.path.basename(cfg_path_cp1251)} (кодировка cp1251):")
freq_net_cp, freq_rate_cp = organizer.extract_frequencies_from_cfg(cfg_path_cp1251, encoding='cp1251')
print(f"  Частота сети: {freq_net_cp} Гц, Частота дискретизации: {freq_rate_cp} Гц (Ожидается: 50, 1500)")

## 4. Группировка файлов по частотам (`group_files_by_frequency_and_rate`)

Этот метод сканирует указанную директорию, извлекает частоты из `.cfg` файлов и перемещает соответствующие пары `.cfg` и `.dat` в поддиректории вида `f_network = X and f_rate = Y`.

In [ ]:
print(f"\nЗапуск группировки файлов в директории: {notebook_source_dir}")
organizer.group_files_by_frequency_and_rate(notebook_source_dir, is_print_message=True, show_progress=True)

print("\nСтруктура директории после группировки:")
for root, dirs, files in os.walk(notebook_source_dir):
    level = root.replace(notebook_source_dir, '').count(os.sep)
    indent = ' ' * 4 * (level)
    print(f"{indent}{os.path.basename(root)}/")
    sub_indent = ' ' * 4 * (level + 1)
    for f_name in files:
        print(f"{sub_indent}{f_name}")

print("\nПроверка оставшихся файлов в корневой директории источника (должны остаться только те, что не удалось сгруппировать):")
for item in os.listdir(notebook_source_dir):
    if os.path.isfile(os.path.join(notebook_source_dir, item)):
        print(f"- {item}")

## 5. Создание инвентарного CSV с хешами `.dat` файлов (`generate_dat_hash_inventory`)

Этот метод работает с директорией, уже структурированной по частотам (как после `group_files_by_frequency_and_rate`). Он создает CSV-файл с информацией о каждом `.dat` файле, включая его хеш-сумму.

In [ ]:
inventory_csv_path = os.path.join(notebook_base_temp_dir, "dat_inventory.csv")

print(f"\nСоздание инвентарного CSV для директории: {notebook_source_dir}")
organizer.generate_dat_hash_inventory(notebook_source_dir, inventory_csv_path, show_progress=True)

if os.path.exists(inventory_csv_path):
    print(f"\nСодержимое инвентарного CSV ({inventory_csv_path}):")
    try:
        inventory_df = pd.read_csv(inventory_csv_path)
        if pd.__version__ > '1.0.0':
            try:
                from IPython.display import display
                display(inventory_df)
            except ImportError:
                print(inventory_df)
        else:
            print(inventory_df)
    except Exception as e:
        print(f"Ошибка чтения CSV: {e}")
else:
    print(f"Инвентарный CSV не был создан (возможно, не было подходящих поддиректорий или .dat файлов).")

In [ ]:
# Очистка временной директории
try:
    if os.path.exists(notebook_base_temp_dir):
        shutil.rmtree(notebook_base_temp_dir)
        print(f"\nВременная директория {notebook_base_temp_dir} удалена.")
    else:
        print(f"\nВременная директория {notebook_base_temp_dir} не найдена для удаления.")
except Exception as e:
    print(f"Ошибка при удалении временной директории {notebook_base_temp_dir}: {e}")